## Regression

In this lecture, I will show you how easy and practical is to engineer the features of a dataset utilising the new package feature engine and the scikit-learn pipeline.

I will skip the bits on data analysis, and I will jump on straight to feature engineering using the feature_engine package.

For the demo, I will use the house price dataset, available in [Kaggle](https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data)

## House Prices dataset

In [1]:
# to handle datasets
import pandas as pd
import numpy as np

# for plotting
import matplotlib.pyplot as plt
% matplotlib inline

# to divide train and test set
from sklearn.model_selection import train_test_split

# feature scaling
from sklearn.preprocessing import MinMaxScaler

# to build the models
from sklearn.linear_model import Lasso

# to evaluate the models
from sklearn.metrics import mean_squared_error
from math import sqrt

# to build the models
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
import xgboost as xgb

pd.pandas.set_option('display.max_columns', None)

from feature_engine import missing_data_imputers as msi
from feature_engine import discretisers as dsc
from feature_engine import categorical_encoders as ce

# sklearn pipeline to put it all together
from sklearn.pipeline import Pipeline as pipe

### Load Datasets

In [2]:
# load dataset
data = pd.read_csv('houseprice.csv')
print(data.shape)
data.head()

(1460, 81)


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


Id is a unique identifier for each of the houses. Thus this is not a variable that we can use.

### Make lists of variables

In [3]:
# categorical variables
categorical = [var for var in data.columns if data[var].dtype=='O']

# make a list of all numerical variables first
numerical = [var for var in data.columns if data[var].dtype!='O']

# temporal variables
year_vars = [var for var in numerical if 'Yr' in var or 'Year' in var]

# discrete variables
discrete = [var for var in numerical if var not in year_vars and len(data[var].unique())<20]

# continuous vars
numerical = [var for var in numerical if var not in discrete and var not in ['Id', 'SalePrice'] and var not in year_vars]

print('There are {} categorical variables'.format(len(categorical)))
print('There are {} discrete variables'.format(len(discrete)))
print('There are {} numerical and continuous variables'.format(len(numerical)))

There are 43 categorical variables
There are 14 discrete variables
There are 18 numerical and continuous variables


### Separate train and test set

In [4]:
# Let's separate into train and test set

X_train, X_test, y_train, y_test = train_test_split(data, data.SalePrice, test_size=0.1,
                                                    random_state=0)
X_train.shape, X_test.shape

((1314, 81), (146, 81))

#### Temporal variables

In [5]:
# function to calculate elapsed time

def elapsed_years(df, var):
    # capture difference between year variable and year the house was sold
    df[var] = df['YrSold'] - df[var]
    return df

In [6]:
for var in ['YearBuilt', 'YearRemodAdd', 'GarageYrBlt']:
    X_train = elapsed_years(X_train, var)
    X_test = elapsed_years(X_test, var)

C:\Users\Sole\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [7]:
X_train[['YearBuilt', 'YearRemodAdd', 'GarageYrBlt']].head()

,YearBuilt,YearRemodAdd,GarageYrBlt
930,2,2,2.0
656,49,2,49.0
45,5,5,5.0
1348,9,9,9.0
55,44,44,44.0


Instead of years, now we have the amount of years passed since the house was built or remodeled and the house was sold. Next, we drop the YrSold variable from the datasets, because we already extracted its value.

In [8]:
# drop YrSold
X_train.drop('YrSold', axis=1, inplace=True)
X_test.drop('YrSold', axis=1, inplace=True)

# remove YrSold from our temporal var list
year_vars.remove('YrSold')

C:\Users\Sole\Anaconda3\lib\site-packages\pandas\core\frame.py:3694: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


### Feature engineering with feature engine and the sklearn pipeline

I will follow the exact same engineering steps from the previous notebook.

The only thing that we need to do, is list, one after the other, the engineering steps in the sklearn pipeline as follows:

In [9]:
price_pipe = pipe([
    # add a binary variable to indicate missing information for the 2 variables below
    ('continuous_var_imputer', msi.AddNaNBinaryImputer(variables = ['LotFrontage', 'GarageYrBlt'])),
     
    # replace NA by the median in the 3 variables below, they are numerical
    ('continuous_var_median_imputer', msi.MeanMedianImputer(imputation_method='median', variables = ['LotFrontage', 'GarageYrBlt', 'MasVnrArea'])),
     
    # replace NA by adding the label "Missing" in categorical variables (transformer will skip those variables where there is no NA)
    ('categorical_imputer', msi.CategoricalVariableImputer(variables = categorical)),
     
    # there were a few variables in the submission dataset that showed NA, but these variables did not show NA in the train set.
    # to handle those, I will add an additional step here
    ('additional_median_imputer', msi.MeanMedianImputer(imputation_method='median', variables = numerical)),

    # disretise numerical variables using trees
    ('numerical_tree_discretiser', dsc.DecisionTreeDiscretiser(cv = 3, scoring='neg_mean_squared_error', variables = numerical, regression=True)),
     
    # remove rare labels in categorical and discrete variables
    ('rare_label_encoder', ce.RareLabelCategoricalEncoder(tol = 0.03, n_categories=1, variables = categorical+discrete)),
     
    # encode categorical variables using the target mean 
    ('categorical_encoder', ce.MeanCategoricalEncoder(variables = categorical+discrete))
     ])

In [12]:
## the categorical encoders only work with categorical variables
# therefore we need to cast the discrete variables into categorical

X_train[discrete]= X_train[discrete].astype('O')
X_test[discrete]= X_test[discrete].astype('O')


In [13]:
# let's create a list of the training variables
training_vars = [var for var in X_train.columns if var not in ['Id', 'SalePrice']]

print('total number of variables to use for training: ', len(training_vars))

total number of variables to use for training:  78


In [14]:
price_pipe.fit(X_train[training_vars], y_train)

Pipeline(steps=[('continuous_var_imputer', AddNaNBinaryImputer(variables=['LotFrontage', 'GarageYrBlt'])), ('continuous_var_median_imputer', MeanMedianImputer(imputation_method='median',
         variables=['LotFrontage', 'GarageYrBlt', 'MasVnrArea'])), ('categorical_imputer', CategoricalVariableImputer(vari...'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageCars', 'PoolArea', 'MoSold']))])

In [15]:
# let's capture the id to add it later to our submission
submission_id = submission['Id']

In [16]:
X_train = price_pipe.transform(X_train[training_vars])
X_test = price_pipe.transform(X_test[training_vars])
submission = price_pipe.transform(submission[training_vars])

In [17]:
# let's check that we didn't introduce NA
len([var for var in training_vars if X_train[var].isnull().sum()>0])

0

In [18]:
# let's check that we didn't introduce NA
len([var for var in training_vars if X_test[var].isnull().sum()>0])

0

In [19]:
# let's check that we didn't introduce NA
len([var for var in training_vars if submission[var].isnull().sum()>0])

0

### Feature scaling

The transformed datasets contain new variables now, because we added binary variables to indicate missing information. So we need to select again our training variables:

In [20]:
training_vars = [var for var in X_train.columns]
len(training_vars)

80

In [21]:
# these are the binary variables that we introduced during feature engineering
[ var for var in training_vars if '_na' in var]

['LotFrontage_na', 'GarageYrBlt_na']

In [22]:
# fit scaler
scaler = MinMaxScaler() # create an instance
scaler.fit(X_train[training_vars]) #  fit  the scaler to the train set for later use

MinMaxScaler(copy=True, feature_range=(0, 1))

The scaler is now ready, we can use it in a machine learning algorithm when required. See below.

### Machine Learning algorithm building

**Note**

The distribution of SalePrice is also skewed, so I will fit the model to the log transformation of the house price.

Then, to evaluate the models, we need to convert it back to prices.

#### xgboost

In [23]:
xgb_model = xgb.XGBRegressor()

eval_set = [(X_test[training_vars], np.log(y_test))]
xgb_model.fit(X_train[training_vars], np.log(y_train), eval_set=eval_set, verbose=False)

pred = xgb_model.predict(X_train[training_vars])
print('xgb train mse: {}'.format(mean_squared_error(y_train, np.exp(pred))))
print('xgb train rmse: {}'.format(sqrt(mean_squared_error(y_train, np.exp(pred)))))
print()
pred = xgb_model.predict(X_test[training_vars])
print('xgb test mse: {}'.format(mean_squared_error(y_test, np.exp(pred))))
print('xgb train rmse: {}'.format(sqrt(mean_squared_error(y_test, np.exp(pred)))))

xgb train mse: 411750478.1958386
xgb train rmse: 20291.635670784122

xgb test mse: 1248980709.6125262
xgb train rmse: 35340.921176626485


This model shows some over-fitting. Compare the rmse for train and test.

#### Random Forests

In [24]:
rf_model = RandomForestRegressor(n_estimators=800, max_depth=6)
rf_model.fit(X_train[training_vars], np.log(y_train))

pred = rf_model.predict(X_train[training_vars])
print('rf train mse: {}'.format(mean_squared_error(y_train, np.exp(pred))))
print('rf train rmse: {}'.format(sqrt(mean_squared_error(y_train, np.exp(pred)))))

print()
pred = rf_model.predict(X_test[training_vars])
print('rf test mse: {}'.format(mean_squared_error(y_test, np.exp(pred))))
print('rf train rmse: {}'.format(sqrt(mean_squared_error(y_test, np.exp(pred)))))

rf train mse: 600781164.2438939
rf train rmse: 24510.837689558753

rf test mse: 1404834603.1085901
rf train rmse: 37481.123290378986


This model shows some over-fitting. Compare the rmse for train and test.

#### Support vector machine

In [25]:
SVR_model = SVR()
SVR_model.fit(scaler.transform(X_train[training_vars]), np.log(y_train))

pred = SVR_model.predict(X_train[training_vars])
print('SVR train mse: {}'.format(mean_squared_error(y_train, np.exp(pred))))
print('SVR train rmse: {}'.format(sqrt(mean_squared_error(y_train, np.exp(pred)))))

print()
pred = SVR_model.predict(X_test[training_vars])
print('SVR test mse: {}'.format(mean_squared_error(y_test, np.exp(pred))))
print('SVR train rmse: {}'.format(sqrt(mean_squared_error(y_test, np.exp(pred)))))

SVR train mse: 6893198624.090748
SVR train rmse: 83025.28906357838

SVR test mse: 7462041362.741304
SVR train rmse: 86383.10808683203


#### Regularised linear regression

In [26]:
lin_model = Lasso(random_state=2909, alpha=0.005)
lin_model.fit(scaler.transform(X_train[training_vars]), np.log(y_train))

pred = lin_model.predict(scaler.transform(X_train[training_vars]))
print('Lasso Linear Model train mse: {}'.format(mean_squared_error(y_train, np.exp(pred))))
print('Lasso Linear Model train rmse: {}'.format(sqrt(mean_squared_error(y_train, np.exp(pred)))))

print()
pred = lin_model.predict(scaler.transform(X_test[training_vars]))
print('Lasso Linear Model test mse: {}'.format(mean_squared_error(y_test, np.exp(pred))))
print('Lasso Linear Model train rmse: {}'.format(sqrt(mean_squared_error(y_test, np.exp(pred)))))

Lasso Linear Model train mse: 1141198152.8570888
Lasso Linear Model train rmse: 33781.62448517076

Lasso Linear Model test mse: 1564309979.8790731
Lasso Linear Model train rmse: 39551.358761477124


The best model is the Lasso, so I will submit only that one for Kaggle.

### Submission to Kaggle

In [27]:
# make predictions for the submission dataset
final_pred = pred = lin_model.predict(scaler.transform(submission[training_vars]))

In [28]:
temp = pd.concat([submission_id, pd.Series(np.exp(final_pred))], axis=1)
temp.columns = ['Id', 'SalePrice']
temp.head()

,Id,SalePrice
0,1461,114441.226248
1,1462,133481.140066
2,1463,205078.922503
3,1464,203355.657256
4,1465,155992.304115


In [29]:
temp.to_csv('submit_housesale.csv', index=False)